In [1]:
import galsim
import numpy as np
import matplotlib.pyplot as plt
from torch.distributions import Uniform, Beta, Normal
import pyccl
import torch

In [2]:
dist = Normal(torch.tensor([0.8159]), torch.tensor([0.001]))
sigma_8 = dist.sample().item()
print(sigma_8)

0.8161540031433105


In [3]:
# cosmology = pyccl.cosmology.Cosmology(sigma8=sigma_8,
#                                       Omega_c=0.25,
#                                       Omega_b=0.0486,
#                                       h=67.7, # order of magnitude is wrong
#                                       n_s=1,
# )
cosmology = pyccl.cosmology.CosmologyVanillaLCDM()

In [4]:
cosmology.compute_linear_power()
power_spectrum = cosmology.get_linear_power()

In [5]:
# power_spectrum.get_spline_arrays[2] returns P(k, a), but we want k for a fixed a

power_spectrum_vals_pk = power_spectrum.get_spline_arrays()[2][0]
power_spectrum_vals_k = power_spectrum.get_spline_arrays()[1]

power_spectrum_vals_pk = np.array([power_spectrum_vals_pk]).T
power_spectrum_vals_k = np.array([power_spectrum_vals_k]).T
print(power_spectrum_vals_pk.shape)

with open('./sample_ps_vals.txt', 'w') as f:
    print(power_spectrum_vals_k.shape, power_spectrum_vals_pk.shape)
    x = np.hstack((power_spectrum_vals_k, power_spectrum_vals_pk))
    print(x.shape)
    np.savetxt(f, x)

(193, 1)
(193, 1) (193, 1)
(193, 2)


In [11]:
lookup = galsim.LookupTable(x=power_spectrum_vals_k, f=power_spectrum_vals_pk, x_log=False, f_log=False)

In [12]:
galsim_ps = galsim.PowerSpectrum(lookup, units=galsim.radians)

In [14]:
grid_size = 10.  # Define the total grid extent, in degrees
ngrid = 20      # Define the number of grid points in each dimension: (ngrid x ngrid)
n_ell = 15  

g1, g2, kappa = galsim_ps.buildGrid(grid_spacing=grid_size/ngrid, ngrid=ngrid, units=galsim.degrees, get_convergence=True)

fig, (one, two) = plt.subplots(nrows=1,ncols=2)
_ = one.imshow(g1)
_ = two.imshow(g2)

GalSimRangeError: x value(s) above the range of the LookupTable. Value [[ 36.          36.          72.         108.         144.
  180.         216.         252.         288.         324.
  360.        ]
 [ 36.          50.91168825  80.49844719 113.84199577 148.43180252
  183.56470249 218.97945109 254.55844123 290.24127894 325.99386497
  361.79552236]
 [ 72.          80.49844719 101.82337649 129.79984592 160.99689438
  193.86593306 227.68399153 262.08395601 296.86360504 331.90360046
  367.12940498]
 [108.         113.84199577 129.79984592 152.73506474 180.
  209.91426821 241.49534157 274.16783181 307.58413483 341.5259873
  375.85103432]
 [144.         148.43180252 160.99689438 180.         203.64675298
  230.51247255 259.59969183 290.24127894 321.99378876 354.55888086
  387.73186611]
 [180.         183.56470249 193.86593306 209.91426821 230.51247255
  254.55844123 281.16898833 309.68370961 339.62332075 370.64268508
  402.49223595]
 [216.         218.97945109 227.68399153 241.49534157 